In [1]:
%cd src/

from pathlib import Path
import numpy as np
import math
from itertools import groupby
import h5py
import numpy as np
import unicodedata
import cv2
import torch
from torch import nn
from torchvision.models import resnet50, resnet101
from torch.autograd import Variable
import torchvision
from data import preproc as pp
from data import evaluation
from torch.utils.data import Dataset
import time
from pytorch_lightning.core.lightning import LightningModule
import pytorch_lightning as pl 
import clip as openai
from PIL import Image
from clip.clip import _transform as preprocess


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=128):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class OCR(nn.Module):

    def __init__(self, vocab_len, hidden_dim, nheads,
                 num_encoder_layers, num_decoder_layers):
        super().__init__()
    
        self.backbone, _ = openai.load("RN50x16", device="cpu")
        del self.backbone.transformer, self.backbone.visual.attnpool
        _ = self.backbone.to(device)
        for name,p in self.backbone.named_parameters():
            if "visual" not in name or "attnpool" in name:
                p.requires_grad =  False
            elif "visual" in name and "bn" in name:
                p.requires_grad =  False

        # create a default PyTorch transformer
        # create conversion layer
        self.conv = nn.Conv2d(3072, hidden_dim, 1)
        self.gelu = nn.GELU()
        self.norm = nn.LayerNorm(hidden_dim)
        decoder_layer = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=nheads)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        
        # prediction heads with length of vocab
        # DETR used basic 3 layer MLP for output
        self.vocab = nn.Linear(hidden_dim,vocab_len)

        # output positional encodings (object queries)
        self.decoder = nn.Embedding(vocab_len, hidden_dim)
        self.query_pos = PositionalEncoding(hidden_dim, .2)

        # spatial positional encodings, sine positional encoding can be used.
        # Detr baseline uses sine positional encoding.
        self.trg_mask = None
  
    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz), 1)
        mask = mask.masked_fill(mask==1, float('-inf'))
        return mask

    def get_feature(self, x):
        x = self.backbone.visual.conv1(x)
        x = self.backbone.visual.bn1(x)   
        x = self.backbone.visual.relu(x)
        x = self.backbone.visual.conv2(x)
        x = self.backbone.visual.bn2(x)   
        x = self.backbone.visual.relu(x)
        x = self.backbone.visual.conv3(x)
        x = self.backbone.visual.bn3(x)   
        x = self.backbone.visual.relu(x)

        x = self.backbone.visual.avgpool(x)
        x = self.backbone.visual.layer1(x)
        x = self.backbone.visual.layer2(x)
        x = self.backbone.visual.layer3(x)
        x = self.backbone.visual.layer4(x)
        return x


    def make_len_mask(self, inp):
        return (inp == 0).transpose(0, 1)


    def forward(self, inputs, trg):
        # propagate inputs through ResNet-101 up to avg-pool layer
        x = self.get_feature(inputs)

        # convert from 2048 to 256 feature planes for the transformer
        h = self.conv(x)
        h = self.gelu(h)
        h = self.norm(h.flatten(2).permute(0,2,1))


        # generating subsequent mask for target
        if self.trg_mask is None or self.trg_mask.size(0) != len(trg):
            self.trg_mask = self.generate_square_subsequent_mask(trg.shape[1]).to(trg.device)

        # Padding mask
        trg_pad_mask = self.make_len_mask(trg)

        # Getting postional encoding for target
        trg = self.decoder(trg)
        trg = self.query_pos(trg)
        output = self.transformer_decoder(trg.permute(1,0,2), h.permute(1,0,2), tgt_mask=self.trg_mask, 
                                  tgt_key_padding_mask=trg_pad_mask.permute(1,0))

        return self.vocab(output.transpose(0,1))


def make_model(vocab_len, hidden_dim=256, nheads=4,
                 num_encoder_layers=4, num_decoder_layers=4):
    
    return OCR(vocab_len, hidden_dim, nheads,
                 num_encoder_layers, num_decoder_layers)

"""
Uses generator functions to supply train/test with data.
Image renderings and text are created on the fly each time.
"""

class DataGenerator(Dataset):
    """Generator class with data streaming"""

    def __init__(self, source, split, transform, tokenizer):
        self.tokenizer = tokenizer
        self.transform = transform
        
        self.split = split
        self.dataset = dict()

        with h5py.File(source, "r") as f:
            self.dataset[self.split] = dict()

            self.dataset[self.split]['dt'] = np.array(f[self.split]['dt'])
            self.dataset[self.split]['gt'] = np.array(f[self.split]['gt'])
          
            randomize = np.arange(len(self.dataset[self.split]['gt']))
            np.random.seed(42)
            np.random.shuffle(randomize)

            self.dataset[self.split]['dt'] = self.dataset[self.split]['dt'][randomize]
            self.dataset[self.split]['gt'] = self.dataset[self.split]['gt'][randomize]

            # decode sentences from byte
            self.dataset[self.split]['gt'] = [x.decode() for x in self.dataset[self.split]['gt']]
            
        self.size = len(self.dataset[self.split]['gt'])


    def __getitem__(self, i):
        img = self.dataset[self.split]['dt'][i]
        
        #making image compatible with resnet
        img = np.repeat(img[..., np.newaxis],3, -1).astype("float32")   
#         img = pp.normalization(img).astype("float32")

        if self.transform is not None:
            aug = self.transform(image=img)
            img = aug['image']
            
#             img = self.transform(img)
            
        y_train = self.tokenizer.encode(self.dataset[self.split]['gt'][i]) 
        
        #padding till max length
        y_train = np.pad(y_train, (0, self.tokenizer.maxlen - len(y_train)))

        gt = torch.Tensor(y_train)

        return img, gt          

    def __len__(self):
      return self.size

class Tokenizer():
    """Manager tokens functions and charset/dictionary properties"""

    def __init__(self, chars, max_text_length=128):
        self.PAD_TK, self.UNK_TK,self.SOS,self.EOS = "¶", "¤", "SOS", "EOS"
        self.chars = [self.PAD_TK] + [self.UNK_TK ]+ [self.SOS] + [self.EOS] +list(chars)
        self.PAD = self.chars.index(self.PAD_TK)
        self.UNK = self.chars.index(self.UNK_TK)

        self.vocab_size = len(self.chars)
        self.maxlen = max_text_length

    def encode(self, text):
        """Encode text to vector"""
        text = unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("ASCII").lower()
        text = " ".join(text.split())

        groups = ["".join(group) for _, group in groupby(text)]
        text = "".join([self.UNK_TK.join(list(x)) if len(x) > 1 else x for x in groups])
        encoded = []

        text = ['SOS'] + list(text) + ['EOS']
        for item in text:
            index = self.chars.index(item)
            index = self.UNK if index == -1 else index
            encoded.append(index)

        return np.asarray(encoded)

    def decode(self, text):
        """Decode vector to text"""
        
        decoded = "".join([self.chars[int(x)] for x in text if x > -1])
        decoded = self.remove_tokens(decoded)
        decoded = pp.text_standardize(decoded)

        return decoded

    def remove_tokens(self, text):
        """Remove tokens (PAD) from text"""

        return text.replace(self.PAD_TK, "").replace(self.UNK_TK, "")

import os
import datetime
import string

batch_size = 8
epochs = 200

# define paths
#change paths accordingly
source = 'iam'
source_path = '../data/{}.hdf5'.format(source)
# define input size, number max of chars per line and list of valid chars
input_size = (1024, 128, 1)
max_text_length = 128
# charset_base = string.printable[:95]
charset_base = string.printable[:36].lower() + string.printable[36+26:95].lower() 

device = torch.device("cuda:3")

# transform = T.Compose([
#     T.ToTensor()])
tokenizer = Tokenizer(charset_base)
import albumentations
import albumentations.pytorch


# transform_train = albumentations.Compose([
#     albumentations.OneOf(
#         [
#             albumentations.MotionBlur(p=1, blur_limit=8),
#             albumentations.OpticalDistortion(p=1, distort_limit=0.05),
#             albumentations.GaussNoise(p=1, var_limit=(10.0, 100.0)),
#             albumentations.RandomBrightnessContrast(p=1, brightness_limit=0.2),
#             albumentations.Downscale(p=1, scale_min=0.3, scale_max=0.5),
#         ],
#         p=.5,
#     ),    
#     albumentations.Normalize(),
#     albumentations.pytorch.ToTensorV2()

# ])

transform_valid = albumentations.Compose(
    [
        albumentations.Normalize(),
        albumentations.pytorch.ToTensorV2()
    ]
)

/home/mhamdan/seq2seqAttenHTR/Transformer_ocr/src


In [8]:
model = make_model( vocab_len=tokenizer.vocab_size,hidden_dim=256, nheads=4,
                 num_encoder_layers=4, num_decoder_layers=6)
_ = model.to(device)

In [9]:
# def plot_grad_flow(named_parameters):
#     ave_grads = []
#     layers = []
#     for n, p in named_parameters:
#         if(p.requires_grad) and ("bias" not in n):
#             layers.append(n)
#             ave_grads.append(p.grad.abs().mean().cpu())
#     plt.plot(ave_grads, alpha=0.3, color="b")
#     plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
#     plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
#     plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.xlabel("Layers")
#     plt.ylabel("average gradient")
#     plt.title("Gradient flow")
#     plt.grid(True)

In [24]:
d = torch.load("../output/iam/checkpoint_weights_iam_small_clip_resnet50x16_aug_decoder_only.hdf5", map_location={"cuda:0":"cuda:3"})

f = {}
for i in d:
    f[i.replace("module.","")] = d[i]

model.load_state_dict(f)

<All keys matched successfully>

In [27]:
def get_memory(model,imgs):
    x = model.get_feature(imgs)
    h = model.conv(x)
    h = model.gelu(h)
    h = model.norm(h.flatten(2).permute(0,2,1))    
    return h.permute(1,0,2)
    

def test(model, test_loader, max_text_length):
    model.eval()
    predicts = []
    gt = []
    imgs = []
    c=0
    with torch.no_grad():
        for batch in test_loader:
            src, trg = batch
            imgs.append(src.flatten(0,1))
            src, trg = src.to(device), trg.to(device)            
            memory = get_memory(model,src.float())
            out_indexes = [tokenizer.chars.index('SOS'), ]
            for i in range(max_text_length):
                mask = model.generate_square_subsequent_mask(i+1).to(device)
                
                trg_tensor = torch.LongTensor(out_indexes).unsqueeze(1).to(device)
                output = model.vocab(model.transformer_decoder(model.query_pos(model.decoder(trg_tensor)), memory,tgt_mask=mask))
                out_token = output.argmax(2)[-1].item()
#                 print(output.argmax(2))
                out_indexes.append(out_token)
                if out_token == tokenizer.chars.index('EOS'):
                    break
            
            predicts.append(tokenizer.decode(out_indexes))
            gt.append(tokenizer.decode(trg.flatten(0,1)))
#             if c==100:
#                 break
            c+=1
    return predicts, gt, imgs

In [20]:
test_loader = torch.utils.data.DataLoader(DataGenerator(source_path,'test',transform_valid, tokenizer), batch_size=1, shuffle=False, num_workers=6)

In [28]:
predicts, gt, imgs = test(model, test_loader, max_text_length)

In [29]:
predicts = list(map(lambda x : x.replace('SOS','').replace('EOS',''),predicts))
gt = list(map(lambda x : x.replace('SOS','').replace('EOS',''),gt))

evaluate = evaluation.ocr_metrics(predicts=predicts,
                                  ground_truth=gt,)

In [30]:
evaluate

array([0.08694478, 0.23839992, 0.83438596])

In [12]:
a = torch.rand(1,3,224,224).cuda()
b = torch.rand(1,128).cuda().long()

In [17]:
c = torch.LongTensor(tokenizer.encode(gt[0])).cuda().unsqueeze(0)

In [18]:
output = model(imgs[0].unsqueeze(0).cuda(),c)

In [19]:
tokenizer.decode(output.argmax(2)[-1])

'tomeuiti opitle ah tnpeanne , ii ae atould boerEOSdpEOS'

In [40]:
tokenizer.decode(output.log_softmax(-1).contiguous().view(-1, tokenizer.vocab_size).argmax(1))

'ttm uati ositoe mh tnpeanngtii terstould bielsnp'